In [ ]:
import os
import pandas as pd

# Extraer datos

BASE_PATH = "../data/processed"
TRAIN_PATHS = [
    "X_train_con_outliers.xlsx",
    "X_train_sin_outliers.xlsx",
    "X_train_con_outliers_norm_standard.xlsx",
    "X_train_sin_outliers_norm_standard.xlsx",
    "X_train_con_outliers_norm_minmax.xlsx",
    "X_train_sin_outliers_norm_minmax.xlsx"
]

TRAIN_DATASETS = []
for path in TRAIN_PATHS:
    TRAIN_DATASETS.append(
        # pd.read_excel(BASE_PATH + "/" + path)
        pd.read_excel(f"{BASE_PATH}/{path}")
        # pd.read_excel(os.path.join(BASE_PATH, path))
    )

TEST_PATHS = [
    "X_test_con_outliers.xlsx",
    "X_test_sin_outliers.xlsx",
    "X_test_con_outliers_norm_standard.xlsx",
    "X_test_sin_outliers_norm_standard.xlsx",
    "X_test_con_outliers_norm_minmax.xlsx",
    "X_test_sin_outliers_norm_minmax.xlsx"
]

TEST_DATASETS = []
for path in TEST_PATHS:
    TEST_DATASETS.append(
        pd.read_excel(f"{BASE_PATH}/{path}")
    )

y_train = pd.read_excel(f"{BASE_PATH}/y_train.xlsx")
y_test = pd.read_excel(f"{BASE_PATH}/y_test.xlsx")


In [7]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import ParameterGrid, GridSearchCV, RandomizedSearchCV
# crea el modelo, entrena el modelo, guardas el target de train y de test. Sacar la precisión para comparar cual es el mejor de todos
results = []
for index, dataset in enumerate(TRAIN_DATASETS):
    print(index)
    model = LogisticRegression(random_state = 42)
    model.fit(dataset, y_train)
    y_pred_train = model.predict(dataset)   # x_train
    y_pred_test = model.predict(TEST_DATASETS[index])   # x_text

    results.append(
        {
            "train": accuracy_score(y_train, y_pred_train),
            "test": accuracy_score(y_test, y_pred_test)
        }
    )
    # print tiempo que ha tardado
results

0


c:\Users\mamen\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\mamen\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\linear_model\_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\mamen\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:1408: DataConversionWarning: A column-vector y was passed w

1


c:\Users\mamen\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\linear_model\_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\mamen\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\mamen\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:1408: DataConversionWarning: A column-vector y was passed w

2
3
4
5


[{'train': 0.906314511232544, 'test': 0.9011656143759106},
 {'train': 0.9046144505160899, 'test': 0.8986158329286061},
 {'train': 0.9117182756527019, 'test': 0.9042010684798446},
 {'train': 0.9045233758348512, 'test': 0.8981301602719767},
 {'train': 0.9115361262902246, 'test': 0.90165128703254},
 {'train': 0.9045840922890103, 'test': 0.898251578436134}]

In [8]:
import numpy as np 
import time
from tqdm import tqdm  # Importar tqdm para mostrar la barra de progreso
best_dataset = 2 # Porque de results, el mayor accuracy en train y test es el de la posición 3

    # "C" : np.logspace(-4, 4, 20),
    # "penalty": ["l1", "l2", "elasticnet", None],
    # "solver": ['liblinear','lbfgs', 'saga'],
    # "fit_intercept": [True, False],
    # "dual":[True, False],
    # "class_weight": [None, "balanced"],
    # "multi_class": ['auto','ovr','multinomial'],
    # 'tol': [1e-4, 1e-3, 1e-2, 1e-1],  # Tolerancia de convergencia
    # 'max_iter': [100, 500, 1000],  # Número máximo de iteraciones
    # 'warm_start': [True, False],  # Continuar desde la última solución
    # 'l1_ratio': np.linspace(0, 1, 5)  # Solo para 'elasticnet'
# Hyperparametrización
hyperparams = {
    "C" : np.logspace(-4, 4, 20),
    "penalty": ["l1", "l2", "elasticnet", None],
    "solver": ['liblinear','lbfgs', 'saga'],
    "fit_intercept": [True, False],
    "dual":[True, False],
    "class_weight": [None, "balanced"],
    "multi_class": ['auto','ovr','multinomial'],
    'tol': [1e-4, 1e-3, 1e-2, 1e-1],  # Tolerancia de convergencia
    'max_iter': [100, 500, 1000],  # Número máximo de iteraciones
    'warm_start': [True, False],  # Continuar desde la última solución
    'l1_ratio': np.linspace(0, 1, 5)  # Solo para 'elasticnet'
}
# inicializar el modelo
model = LogisticRegression(random_state = 42)   
# se decide hacer el RandomizedSearchCV en vez de GridSearchCV
grid = RandomizedSearchCV(  #  GridSearchCV con este sería mejor pero la estimación de tiempo es grande 6h-36h
    model, 
    hyperparams, 
    scoring = "accuracy", 
    n_iter = 100,
    cv = 5,
    n_jobs = -1,
    verbose = 2,   #este parámetro mostrará info sobre cada iteración
    random_state = 42
    ) # n_jobs=2 hacerlo más rápido dependiendo los núcleos del ordenador
grid

RandomizedSearchCV(cv=5, estimator=LogisticRegression(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'C': array([1.00000000e-04, 2.63665090e-04, 6.95192796e-04, 1.83298071e-03,
       4.83293024e-03, 1.27427499e-02, 3.35981829e-02, 8.85866790e-02,
       2.33572147e-01, 6.15848211e-01, 1.62377674e+00, 4.28133240e+00,
       1.12883789e+01, 2.97635144e+01, 7.84759970e+01, 2....
                                        'class_weight': [None, 'balanced'],
                                        'dual': [True, False],
                                        'fit_intercept': [True, False],
                                        'l1_ratio': array([0.  , 0.25, 0.5 , 0.75, 1.  ]),
                                        'max_iter': [100, 500, 1000],
                                        'multi_class': ['auto', 'ovr',
                                                        'multinomial'],
                                        'penalty': ['l1', 'l2', 'elasticnet',
                                                    None],
                                        'solver': ['liblinear', 'lbfgs',
                                                   'saga'],
                                        'tol': [0.0001, 0.001, 0.01, 0.1],
                                        'warm_start': [True, False]},
                   random_state=42, scoring='accuracy', verbose=2)

In [9]:
# entrenar el modelo
grid.fit(TRAIN_DATASETS[best_dataset], y_train) # 36min 50.000 fits
# LogisticRegression(C=np.float64(29.763514416313132), dual=True,
#                    l1_ratio=np.float64(0.25), max_iter=1000, multi_class='auto',
#                    random_state=42, solver='liblinear', tol=0.01)
## 1min 32seg
# LogisticRegression(C=np.float64(10000.0), l1_ratio=np.float64(0.0),
#                    max_iter=1000, multi_class='ovr', random_state=42,
#                    tol=0.001)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


c:\Users\mamen\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\model_selection\_validation.py:528: FitFailedWarning: 
330 fits failed out of a total of 500.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
35 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\mamen\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\model_selection\_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
    ~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\mamen\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\base.py", line 1389, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "c:\Users\mamen\AppD

RandomizedSearchCV(cv=5, estimator=LogisticRegression(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'C': array([1.00000000e-04, 2.63665090e-04, 6.95192796e-04, 1.83298071e-03,
       4.83293024e-03, 1.27427499e-02, 3.35981829e-02, 8.85866790e-02,
       2.33572147e-01, 6.15848211e-01, 1.62377674e+00, 4.28133240e+00,
       1.12883789e+01, 2.97635144e+01, 7.84759970e+01, 2....
                                        'class_weight': [None, 'balanced'],
                                        'dual': [True, False],
                                        'fit_intercept': [True, False],
                                        'l1_ratio': array([0.  , 0.25, 0.5 , 0.75, 1.  ]),
                                        'max_iter': [100, 500, 1000],
                                        'multi_class': ['auto', 'ovr',
                                                        'multinomial'],
                                        'penalty': ['l1', 'l2', 'elasticnet',
                                                    None],
                                        'solver': ['liblinear', 'lbfgs',
                                                   'saga'],
                                        'tol': [0.0001, 0.001, 0.01, 0.1],
                                        'warm_start': [True, False]},
                   random_state=42, scoring='accuracy', verbose=2)

In [16]:
grid.best_estimator_

LogisticRegression(C=np.float64(10000.0), l1_ratio=np.float64(0.0),
                   max_iter=1000, multi_class='ovr', random_state=42,
                   tol=0.001)

In [ ]:
final_model = grid.best_estimator_
# la predicción del modelo
y_pred_test = final_model.predict(TEST_DATASETS[best_dataset])
y_pred_train = final_model.predict(TRAIN_DATASETS[best_dataset])

In [18]:
y_pred_test

array([1, 0, 0, ..., 0, 0, 0], shape=(8236,))

In [19]:
y_pred_train

array([0, 0, 0, ..., 0, 0, 0], shape=(32940,))

Precisión

In [20]:
accuracy_score(y_train, y_pred_train)

0.9119307832422586

In [21]:
accuracy_score(y_test, y_pred_test)

0.9040796503156873

## Metric Train > Metric Test -> Overfitting: El modelo ha aprendido demasiado de train -> El modelo no ha generalizado
    1. Feature selection
    2. El modelo no sea el apropiado
    3. El modelo no está bien optimizado
    4. Reformular train y test
## Metric Train < Metric Test -> Underfitting
    1. Reformular train y test
    2. El modelo no sea el apropiado
## Metric Train ~ Metric Test y los resultados son malos ->
    1. El modelo no sea el apropiado
    2. El modelo no está bien optimizado
    3. (Feature selection)

Se hace el Feature Selection. Pero no mejora, se decide mantener.

In [ ]:
# Se prueba el Feature Selection y no obtenemos mejores resultados. Empeoran al bajarle la k
# Al tratarse de un valor del 91% se considera aceptable para el modelo
base_path = "../data/processed/assets"
train_paths = "clean_with_outliers_with_norm_standard_train.csv"
x_train = pd.read_csv(f"{base_path}/{train_paths}", sep=",")


test_paths = "clean_with_outliers_with_norm_standard_test.csv"
x_test = pd.read_csv(f"{base_path}/{test_paths}", sep=",")

y_train = x_train["y"]
y_test = x_test["y"]

x_train = x_train.drop(columns=["y"])
x_test = x_test.drop(columns=["y"])

In [59]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import ParameterGrid, GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import cross_val_score
import warnings
warnings.filterwarnings("ignore")

results = []
# Medir tiempo de entrenamiento
start = time.time()

model = LogisticRegression(random_state = 42)
model.fit(x_train, y_train)

end = time.time()

y_pred_train = model.predict(x_train)
y_pred_test = model.predict(x_test)

results.append(
    {
        "train": accuracy_score(y_train, y_pred_train),
        "test": accuracy_score(y_test, y_pred_test)
    })
print(results)
cv_scores = cross_val_score(model, x_train, y_train.ravel(), cv=5, scoring="accuracy")
print("Cross-validation accuracy:", cv_scores.mean())
    # print tiempo que ha tardado
# k=13 [{'train': 0.911778992106861, 'test': 0.9039582321515298}]

[{'train': 0.9115361262902246, 'test': 0.9040796503156873}]
Cross-validation accuracy: 0.9114754098360656
